# Implementing RK1, RK2 and RK4 ODE Solvers

## Numerically Solving Higher Order Ordinary Differential Equations

A first order Ordinary Differential Equation (ODE) can be written in the form
$$\frac{dy(t)}{dt} = f(y(t), t)$$
Here, $y$ can be an $\mathbb{R}$-valued or vector-valued function. If $y(t) = (y_1(t), \ldots, y_k(t))$, then $\frac{dy(t)}{dt} = (\frac{dy_1(t)}{dt}, \ldots, \frac{dy_k(t)}{dt})$.

Higher order ODEs can also be converted to a system of first order ODEs and then represented as a single ODE of a vector-valued function. 

### Initial Value Problem (IVP)

An *Initial Value Problem* is an ODE together with the value of $y$ at some point $t_0$. The task then is to find a solution for $y$ in some interval $[a, b]$. (Usually $a = t_0$).

A unique solution is guaranteed if $f$ satisfies certain continuity conditions. [Picard–Lindelöf theorem](https://en.wikipedia.org/wiki/Picard%E2%80%93Lindel%C3%B6f_theorem).

### Numerical Solution

Solving an IVP analytically (symbollically) can be hard or even impossible (?). But there are multiple ways to solve any IVP numerically. The `solve_ivp` function in `numpy.integrate` supports five such methods - RK23, RK45, DOP853, BDF, LSODA, with RK45 being the default.

#### Example 1. Simple harmonic oscillation

\begin{align*}
    \frac{d}{dt} y_1 = y_2 \\
    \frac{d}{dt} y_2 = -y_1
\end{align*}

with initial conditions $y_1(0) = 0$ and $y_2(0) = 1$.

Solve it in the interval $[0,10]$

Note that this differential equation has an exact solution 
$$ y_1(t) = sin(t) $$
which you can use to check the accuracy of your numerical solutions

In [1]:
# Importing packages
from scipy import integrate
import matplotlib.pyplot as plt
import numpy as np
import math as math